In [34]:
import keras
import glob
import os
import xml.etree.ElementTree as ET
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from skimage import transform
import numpy as np
from keras.preprocessing import image
from keras.models import load_model
import shutil

Populating the interactive namespace from numpy and matplotlib


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['load']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [9]:
image1 = '/Volumes/My Passport/CNNData/validation_set/croppedImages/num_1_10293193+Dermacentor_variabilis+Augment+RTS_28+life_a+fed.jpg'
image3 = '/Volumes/My Passport/CNNData/validation_set/croppedImages/num_1_10316471+Ixodes_scapularis+Augment+RTS_28+life_a+fed.jpg'
image4 = '/Volumes/My Passport/CNNData/validation_set/croppedImages/num_1_41227+Ixodes_scapularis+Discard+RTS_+life_unk+feed_unk.jpg' #notick

new_model = load_model('/Volumes/My Passport/CNNData/Weights/VGG16_AI_Tick (1).h5')

def load(np_image):
 # np_image = Image.open(filename)
 np_image = np.array(np_image).astype('float32')/255
 np_image = transform.resize(np_image, (64,64,3))
 np_image = np.expand_dims(np_image, axis=0)
 return np_image

def cnnPred(image_path):

    image_load = load(image_path)
    result = new_model.predict(image_load) 
    if result[0][0] > 0.5:
        return 'no tick'
        # prediction = 'no tick'
    else:
        return 'tick'
        # prediction = 'tick'

    # img2 = mpimg.imread(image)
    # imgplot = plt.imshow(img2)
    # plt.show()

    # print(prediction)


In [32]:
def read_content(xml_file: str):

    tree = ET.parse(xml_file)
    root = tree.getroot()

    list_with_all_boxes = []

    for boxes in root.iter('object'):

        filename = root.find('filename').text

        ymin, xmin, ymax, xmax = None, None, None, None

        ymin = int(float(boxes.find("bndbox/ymin").text))
        xmin = int(float(boxes.find("bndbox/xmin").text))
        ymax = int(float(boxes.find("bndbox/ymax").text))
        xmax = int(float(boxes.find("bndbox/xmax").text))

        list_with_single_boxes = [xmin, ymin, xmax, ymax, filename]
        list_with_all_boxes.append(list_with_single_boxes)

    return list_with_all_boxes

def cropncreateIMG(bnd_list):

    predList = []
    count = 0

    for boxes in bnd_list:

        count += 1

        xmin = boxes[0]
        ymin = boxes[1]
        xmax = boxes[2]
        ymax = boxes[3]

        filename = boxes[4]

        filename = filename.removeprefix('/Volumes/My Passport/originalData/') # **********

        filepath = "/Volumes/My Passport/AutoboxTrial/" + filename 

        im = Image.open(filepath) 
        image = im.crop((xmin, ymin, xmax, ymax))
        
        prediction = cnnPred(image)
        predList.append(prediction)

    anotherCount = 0
    for item in predList:
        if item == 'tick':
            anotherCount+=1 

    if anotherCount == len(predList):
        shutil.copy(filepath, "/Volumes/My Passport/FilteredData/peak/" + filename)
        xmlfilename = filename.removesuffix('.jpg') + '.xml'
        shutil.copy("/Volumes/My Passport/AutoboxTrial/" + xmlfilename, "/Volumes/My Passport/FilteredData/peak/" + xmlfilename)

    # else:
        # remove the boundingboxes that are of no tick
        # save and move resulting xmlfile and jpgfile 



        # return image
        # image.save("/Volumes/My Passport/CNNData/validation_set/croppedImages/" + "num_" + str(count) + "_" + filename)



In [36]:
img1 = "/Volumes/My Passport/CNNData/validation_set/10094333+Amblyomma_americanum+Discard+RTS_50+life_unk+feed_unk.xml"
img2 = "/Volumes/My Passport/CNNData/validation_set/41227+Ixodes_scapularis+Discard+RTS_+life_unk+feed_unk.xml"

testimg1 = "/Volumes/My Passport/AutoboxTrial/10094333+Amblyomma_americanum+Discard+RTS_50+life_unk+feed_unk.xml"

boundingboxList = read_content(testimg1)
returnImg = cropncreateIMG(boundingboxList)
cnnPred(returnImg)

TypeError: expected str, bytes or os.PathLike object, not Image

In [ ]:
folder = '/Volumes/My Passport/AutoboxTrial'
for xmlfile in glob.iglob(os.path.join(folder, "*.xml")):
    
    
    